In [2]:

###############
# 1. import
#####

import pandas as pd
from datetime import datetime, timedelta
import re
import time 

# local only
import os
import cx_Oracle as oci
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

path = 'C:/windows/Fonts/malgun.ttf'                           # 폰트:맑은고딕
font_name = fm.FontProperties(fname=path, size=24).get_name()  
plt.rc('font', family=font_name)


C:\Users\MAEIL\AppData\Roaming\Python\Python37\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [3]:
###############
# 1.데이터 불러오기
#####
# DB접속 : 접속정보 읽기
con_text = open("C:/Users/MAEIL/conn_info_MMBSHIP.txt", 'r', encoding='utf8')
con_text = con_text.read()

# DB 접속 : 실제 접속
conn = oci.connect(con_text[1:], encoding='UTF-8', nencoding='UTF-8')

In [4]:
###############
# 2.데이터 정렬
#####
query = '''
        --포인트 적립/사용 + 기프트카드사용
        select unfy_mmb_no
        , (select code_nm from mmbship.JT_CODE where code_grp_id = 'PM001' and trsc_typ_cd = code) as trsc_typ_cd
        , coopco_cd
        , tot_sel_amt
        , reg_dtm as reg_dtm
        from mmbship.TP_TRSC_PTCL
        where unfy_mmb_no is not null 
        and tot_sel_amt > 0 
        and reg_dtm between ADD_MONTHS( TRUNC(sysdate,'MM'), -25 ) and ADD_MONTHS( LAST_DAY(sysdate), -1 )
        and trsc_typ_cd in ( 'G40', '200', '300')
        union all
        --쿠폰 사용
        select A.unfy_mmb_no
        , '쿠폰사용' as trsc_typ_cd
        , U.coopco_cd
        , u.dc_tgt_amt as tot_sel_amt
        , A.reg_dtm as reg_dtm
        from mmbship.TPM_CPN_ISSU A, mmbship.TPM_CPN B, mmbship.TPM_CPN_use U
        where A.CPN_CD = B.cpn_cd 
        and cpn_st_cd ='2' 
        and A.CPN_issu_CD = U.CPN_issu_CD
        and U.coopco_cd in (7050, 7060)
        and A.unfy_mmb_no is not null
        and A.reg_dtm between ADD_MONTHS( TRUNC(sysdate,'MM'), -25 ) and ADD_MONTHS( LAST_DAY(sysdate), -1 )
        '''

df = pd.read_sql(query, con = conn )

In [23]:
df = df.sort_values(by=['UNFY_MMB_NO', 'REG_DTM'], axis=0)
df = df.reset_index(drop=True)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705386 entries, 0 to 705385
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   UNFY_MMB_NO  705386 non-null  int64         
 1   TRSC_TYP_CD  705386 non-null  object        
 2   COOPCO_CD    705386 non-null  object        
 3   TOT_SEL_AMT  705386 non-null  int64         
 4   REG_DTM      705386 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 26.9+ MB


In [12]:
# df['REG_DTM'] = pd.to_datetime(df['REG_DTM'],format='YY/MM/DD')

In [25]:
df.REG_DTM.max()

Timestamp('2022-01-31 08:50:11')

In [26]:
df.head(10)

,UNFY_MMB_NO,TRSC_TYP_CD,COOPCO_CD,TOT_SEL_AMT,REG_DTM
0,381692,포인트 적립/적립취소,7040,102920,2021-05-01 01:07:33
1,381692,포인트 사용/사용취소,7040,48000,2021-05-13 14:17:34
2,381692,포인트 적립/적립취소,7040,48000,2021-05-18 01:07:28
3,381692,포인트 적립/적립취소,7040,40206,2021-05-28 01:09:47
4,381692,포인트 적립/적립취소,7040,44800,2021-06-27 01:09:34
5,381692,포인트 사용/사용취소,7040,22880,2021-11-02 07:21:16
6,381692,포인트 적립/적립취소,7040,25880,2021-11-06 01:09:44
7,381692,포인트 적립/적립취소,7040,40148,2022-01-07 01:07:06
8,382221,포인트 적립/적립취소,7040,51000,2020-12-14 10:13:01
9,382749,기프트카드사용/사용취소,7030,7000,2021-09-14 12:33:11


In [17]:
dataD = df.copy()
dataD = dataD.sort_values(by=['UNFY_MMB_NO', 'REG_DTM'], axis=0)
dataD = dataD.dropna()

#충전과 충전 사이의 gap에 대한 계산
dataD['DIFF'] = dataD.groupby(['UNFY_MMB_NO'])['REG_DTM'].transform(lambda x: x.diff()).dt.days

In [19]:
dataD.head(10)

,UNFY_MMB_NO,TRSC_TYP_CD,COOPCO_CD,TOT_SEL_AMT,REG_DTM,DIFF
561112,381692,포인트 적립/적립취소,7040,102920,2021-05-01 01:07:33,NaN
548482,381692,포인트 사용/사용취소,7040,48000,2021-05-13 14:17:34,12.0
542690,381692,포인트 적립/적립취소,7040,48000,2021-05-18 01:07:28,4.0
541783,381692,포인트 적립/적립취소,7040,40206,2021-05-28 01:09:47,10.0
577021,381692,포인트 적립/적립취소,7040,44800,2021-06-27 01:09:34,29.0
649865,381692,포인트 사용/사용취소,7040,22880,2021-11-02 07:21:16,128.0
623781,381692,포인트 적립/적립취소,7040,25880,2021-11-06 01:09:44,3.0
679624,381692,포인트 적립/적립취소,7040,40148,2022-01-07 01:07:06,61.0
433637,382221,포인트 적립/적립취소,7040,51000,2020-12-14 10:13:01,NaN
622185,382749,기프트카드사용/사용취소,7030,7000,2021-09-14 12:33:11,NaN


In [20]:
# pd.pivot_table(dataD, values='DIFF', index=['UNFY_MMB_NO'], columns=['COOPCO_CD'],  aggfunc=np.average)

In [20]:
dataD.dropna().groupby(['COOPCO_CD'])['DIFF'].sum() / dataD.dropna().groupby(['COOPCO_CD'])['DIFF'].count()

COOPCO_CD
7010    85.308776
7020    41.638424
7030    30.732539
7040    30.934381
7050    29.383519
7060    19.829121
Name: DIFF, dtype: float64

In [21]:
dataD.dropna().groupby(['COOPCO_CD'])['DIFF'].count()

COOPCO_CD
7010       923
7020    321111
7030    142907
7040     70452
7050      6808
7060      2651
Name: DIFF, dtype: int64

In [22]:
dataD.dropna().groupby(['COOPCO_CD'])['DIFF'].sum()

COOPCO_CD
7010       78740.0
7020    13370556.0
7030     4391895.0
7040     2179389.0
7050      200043.0
7060       52567.0
Name: DIFF, dtype: float64